In [88]:
import numpy as np
import pandas as pd
import names
import random

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler 
from sklearn.metrics import accuracy_score, log_loss
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold, learning_curve

In [125]:
df = pd.read_csv("transfusion.data")
df.columns = ['Recency', 'Frequency', 'Monetary', 'Time', 'Label']

In [126]:
df.head()

,Recency,Frequency,Monetary,Time,Label
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
Recency      748 non-null int64
Frequency    748 non-null int64
Monetary     748 non-null int64
Time         748 non-null int64
Label        748 non-null int64
dtypes: int64(5)
memory usage: 29.3 KB


In [39]:
df.Label.value_counts()

0    570
1    178
Name: Label, dtype: int64

In [127]:
df = pd.read_csv("transfusion.data")
df.columns = ['Recency', 'Frequency', 'Monetary', 'Time', 'Label']

labels = df['Label']
df = df.drop(['Label'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(df, labels, test_size=0.2)


In [128]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)

scaler = StandardScaler().fit(X_test)
X_test = scaler.transform(X_test)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  """


In [129]:
clf = RandomForestClassifier()

'''rf_param_grid = {"max_depth": [80, 90, 100, 110],
              "max_features": [2, 3],
              "min_samples_split": [8, 10, 12],
              "min_samples_leaf": [3, 4, 5],
              "bootstrap": [False],
              "n_estimators" :[100, 200, 300, 1000],
              "criterion": ["gini"]}


gsRFC = GridSearchCV(clf,param_grid = rf_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsRFC.fit(X_train,y_train)

RFC_best = gsRFC.best_estimator_

# Best score
gsRFC.best_score_'''

clf.fit(X_train, y_train)
predictions = clf.predict(X_test)
accuracy = accuracy_score(y_test, predictions) * 100
print("Accuracy: " + str(accuracy) + "%")

Accuracy: 74.0%


/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 4 columns):
Recency      748 non-null int64
Frequency    748 non-null int64
Monetary     748 non-null int64
Time         748 non-null int64
dtypes: int64(4)
memory usage: 23.5 KB


In [91]:
# Gradient boosting tunning
kfold = StratifiedKFold(n_splits=10)

GBC = GradientBoostingClassifier()
gb_param_grid = {'loss' : ["deviance"],
              'n_estimators' : [100,200,300],
              'learning_rate': [0.1, 0.05, 0.01],
              'max_depth': [4, 8],
              'min_samples_leaf': [100,150],
              'max_features': [0.3, 0.1] 
              }

gsGBC = GridSearchCV(GBC, param_grid = gb_param_grid, cv=kfold, scoring="accuracy", n_jobs= 4, verbose = 1)

gsGBC.fit(X_train, y_train)

GBC_best = gsGBC.best_estimator_

# Best score
gsGBC.best_score_

Fitting 10 folds for each of 72 candidates, totalling 720 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    6.5s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:   13.5s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:   24.3s
[Parallel(n_jobs=4)]: Done 720 out of 720 | elapsed:   35.9s finished


0.8010033444816054

In [130]:
predictions = gsGBC.predict(X_test)
accuracy = accuracy_score(y_test, predictions) * 100
print("Accuracy: " + str(accuracy) + "%")

Accuracy: 80.0%


In [131]:
train_labels = gsGBC.predict(X_train)
test_labels = gsGBC.predict(X_test)

In [132]:
output_labels = list(train_labels) + list(test_labels)

In [141]:
output_df['Label'] = labels
output_df.head()

,Recency,Frequency,Monetary,Time,Label
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [226]:
names_list = []
blood_groups_list, phone_numbers = [], []
blood_groups = ["A+", "A-", "B+", "B-", "O+", "O-", "AB+", "AB-"]
lat_values, long_values = [], []

def gen_phone():
    first = str(random.randint(100,999))
    second = str(random.randint(1,888)).zfill(3)

    last = (str(random.randint(1,9998)).zfill(4))
    while last in ['1111','2222','3333','4444','5555','6666','7777','8888']:
        last = (str(random.randint(1,9998)).zfill(4))

    return '{}-{}-{}'.format(first,second, last)

def create_random_point(x0,y0,distance):
    """
            Utility method for simulation of the points
    """   
    r = distance/ 111300
    u = np.random.uniform(0,1)
    v = np.random.uniform(0,1)
    w = r * np.sqrt(u)
    t = 2 * np.pi * v
    x = w * np.cos(t)
    x1 = x / np.cos(y0)
    y = w * np.sin(t)
    return (x0 + x1, y0 +y)

latitude1,longitude1 = 33.4484,-112.0740

for i in range(748):
    blood_groups_list.append(random.choice(blood_groups))
    names_list.append(names.get_full_name())
    phone_numbers.append(gen_phone())
    lat, long = create_random_point(latitude1, longitude1, 15000)
    lat_values.append(lat)
    long_values.append(long)
    

In [227]:
output_df["Name"] = names_list
output_df["Blood Group"]= blood_groups_list
output_df["Phone Number"] = phone_numbers
output_df["Lat"] = lat_values
output_df["Long"] = long_values

In [228]:
output_df.head(10)

,Recency,Frequency,Monetary,Time,Label,Name,Blood Group,Phone Number,Lat,Long
0,2,50,12500,98,1,David Young,B+,743-107-8251,33.515607,-111.958959
1,0,13,3250,28,1,Vincent Cremeans,AB+,367-391-1765,33.580894,-112.150508
2,1,16,4000,35,1,Jason Munoz,AB-,561-723-7137,33.628149,-112.004967
3,2,20,5000,45,1,Becky Oconnor,AB-,257-511-0139,33.525067,-112.070459
4,1,24,6000,77,0,Mickey Chavez,A+,732-421-0985,33.361807,-111.989084
5,4,4,1000,4,0,Amanda Sanchez,O-,748-758-5125,33.422955,-112.155187
6,2,7,1750,14,1,Mark Frye,O-,204-314-6004,33.439548,-112.061050
7,1,12,3000,35,0,Erin Tabor,AB-,842-172-3112,33.628309,-112.138774
8,2,9,2250,22,1,Laverne Nardone,O-,967-295-8605,33.588712,-112.156972
9,5,46,11500,98,1,Jasper Beals,O+,484-287-4215,33.360062,-112.126752


In [229]:
output_df['Blood Group'].value_counts()

B+     107
AB-    105
O-      97
B-      94
A+      94
AB+     91
A-      86
O+      74
Name: Blood Group, dtype: int64

In [230]:
output_df.to_csv("donors.csv", index=False)

In [235]:
modified_df = pd.DataFrame()
modified_df["Name"] = output_df.Name
modified_df["Phone"] = output_df["Phone Number"]
modified_df["Lat"] = output_df["Lat"]
modified_df["Long"] = output_df["Long"]
modified_df["BloodGroup"] = output_df["Blood Group"]
modified_df["Label"] = output_df["Label"]

In [237]:
modified_df.to_csv("table_data.csv", index=False)